# MovieLens: Carga en base de datos

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

In [4]:
import os
import numpy as np
import pandas as pd
import mysql.connector
import logging
from tmdb_api import TMDbApi
from logger import LoggerBuilder

from parallel_pandas import ParallelPandas

## Setup

In [5]:
DATASET_PATH = '../../datasets'
MM_PATH      = f'{DATASET_PATH}/movielens-movies.json'

In [6]:
LoggerBuilder.build()

In [7]:
ParallelPandas.initialize(n_cpu=24, split_factor=4)

## Funciones

In [41]:
def execute(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
    except Exception as error:
        print(error)
        print(f'Query: {query}')


def insert_item(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_item(id, title, overview, image)
        VALUES (
            :ID,
            ":NAME",
            ":DESC",
            ":IMAGE"
        );    
        """.replace(':ID', str(row['id'])) \
            .replace(':NAME', str(row['title'])) \
            .replace(':DESC', str(row['overview']).replace('"', '')) \
            .replace(':IMAGE', str(row['image']))
    execute(connection, query)
        
def insert_interaction(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_interaction(movie_id, user_id, rating)
        VALUES (
            ":ITEM_ID",
            ":USER_ID",
            ":RATING"
        );    
        """.replace(':ITEM_ID', str(int(row['movie_id']))) \
            .replace(':USER_ID', str(int(row['user_id']))) \
            .replace(':RATING', str(row['rating']))
    execute(connection, query)
    
    
def preprocess_movie_metadata(df):
    # To numeric id...
    df['imdb_id'] = df['imdb_id'].apply(lambda x: x.replace('tt', '') if type(x) == str else x)
    
    # Filter not null...
    df = df[~df['imdb_id'].isnull()]
    
    # Filter inconsistents movie ids..
    df = df[~df['id'].str.contains('-')]
    
    # Filter not null...
    df = df[~df['id'].isnull()]
    
    # Make numeric columns...
    df['id']      = df['id'].astype(dtype='int64')
    df['imdb_id'] = df['imdb_id'].astype(dtype='int64')

    # Renameing...
    return df[['id', 'imdb_id']].rename(columns={'id': 'tmdb_id'})

## Procedimiento

**Step 1**: Cargamos los datasets.

In [9]:
movies          = pd.read_json(f'{DATASET_PATH}/movies.json')
interactions    = pd.read_json(f'{DATASET_PATH}/interactions.json')

In [11]:
if os.path.exists(MM_PATH):
    mm_df = pd.read_json(f'{DATASET_PATH}/movielens-movies.json')
else:
    mm_df = pd.read_csv(f'../../thesis-datasets/tmdb/movies_metadata.csv')

    mm_df = preprocess_movie_metadata(mm_df)

    mm_df = movies.merge(mm_df, on='imdb_id')
    mm_df = mm_df[['id', 'imdb_id', 'tmdb_id', 'title', 'overview']]

    api = TMDbApi()
    mm_df['image'] = mm_df['tmdb_id'].p_apply(lambda movie_id: api.movie_by(movie_id).poster)

    mm_df.to_json(MM_PATH)

In [35]:
mm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33467 entries, 0 to 33466
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        33467 non-null  int64 
 1   imdb_id   33467 non-null  int64 
 2   tmdb_id   33467 non-null  int64 
 3   title     33467 non-null  object
 4   overview  33467 non-null  object
 5   image     33016 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.8+ MB


In [36]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210725 entries, 0 to 210724
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   rating     210725 non-null  float64       
 1   timestamp  210725 non-null  datetime64[ns]
 2   user_id    210725 non-null  int64         
 3   movie_id   210725 non-null  int64         
 4   tags       210725 non-null  object        
 5   year       210725 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 11.3+ MB


**Step 2**: Creamos una conexión a la base de datos.

In [12]:
connection = mysql.connector.connect(
  host     = "localhost",
  user     = "root",
  password = "1234"
)

**Step 3**: Borramos la tabla de items para cargar la nueva version de items desde el dataframe de items.

In [28]:
execute(connection, 'DELETE FROM recsys.recsysweb_interaction')

In [29]:
execute(connection, 'ALTER TABLE recsys.recsysweb_item MODIFY id int(11) NOT NULL')

In [30]:
execute(connection, 'DELETE FROM recsys.recsysweb_item')

**Step 4**: Se insertan los nuevos items en la tabla `recsys.recsysweb_item`.

In [ ]:
for _, row in mm_df.iterrows(): insert_item(connection, row)

**Step 5**: Se realiza el mismo procedimiento con la tabla de interacciones (`recsys.recsysweb_interaction`). Se borra la tabla y se vuelve a insertar todas las interacciones.

In [ ]:
for _, row in interactions.iterrows(): insert_interaction(connection, row)

**Step 6**: Se cierra la conexión con la base de datos.

In [43]:
connection.close()

**Step 7**: Se consulta la cantidad de usuario en el dataset de interacciones. 

In [44]:
interactions.user_id.max()

162534